In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/weather_bigdata_contest/code

/content/drive/MyDrive/weather_bigdata_contest/code


In [2]:
import pandas as pd
import datetime as dt
from sklearn.model_selection import train_test_split

In [4]:
sale_df = pd.read_csv('/content/drive/MyDrive/weather_data/sales_data.csv')
weather_df = pd.read_csv('/content/drive/MyDrive/weather_data/weather_final.csv', encoding = 'CP949')

In [5]:
# 병합하려고 했는데 날짜 column name이 달라서 '날짜'로 통일
weather_df.rename(columns = {'일시':'날짜'}, inplace = True)

In [6]:
# pandas의 merge_asof method 사용하여 '날짜'를 기준으로 합치는 방법 사용하려고 했는데, '날짜'의 type이 str이라서 integer나 float으로 변경 필요
# pandas의 to_datetime method를 사용하여 type 변경
sale_df['날짜'] = pd.to_datetime(sale_df['날짜'], format='%Y-%m-%d')
sale_df = sale_df.sort_values('날짜')
weather_df['날짜'] = pd.to_datetime(weather_df['날짜'], format = '%Y-%m-%d')
weather_df = weather_df.sort_values('날짜')

In [7]:
# df 순서를 바꾸면 안됨
sale_and_weather_df = pd.merge_asof(sale_df, weather_df, on='날짜', allow_exact_matches=True)

In [8]:
sale_and_weather_df.head()

,날짜,대분류_냉난방가전,대분류_뷰티,대분류_식품,소분류_가공란,소분류_가스온수기,소분류_가열식 가습기,소분류_가자미,소분류_갈비/찜/바비큐용 돈육,소분류_갈비용 우육,소분류_갈치,소분류_감/홍시,소분류_감귤/한라봉/오렌지,소분류_감마리놀렌산 영양제,소분류_감말랭이,소분류_감자,소분류_갓김치,소분류_건강즙,소분류_건강즙/녹용,소분류_건대추,소분류_건망고,소분류_건바나나,소분류_건어물 건새우,소분류_건어물 노가리,소분류_건어물 마른오징어,소분류_건어물 멸치,소분류_건어물 쥐포,소분류_건어물 진미채,소분류_건어물 황태,소분류_건자두,소분류_건포도,소분류_게장류,소분류_견과류,소분류_견과류 땅콩,소분류_견과류 마카다미아,소분류_견과류 밤,소분류_견과류 잣/은행,소분류_견과류 카카오닙스,소분류_견과류 캐슈넛,소분류_견과류 피스타치오,...,1시간 최다 강수량 시각(hh)9.0,1시간 최다 강수량 시각(hh)10.0,1시간 최다 강수량 시각(hh)11.0,1시간 최다 강수량 시각(hh)12.0,1시간 최다 강수량 시각(hh)13.0,1시간 최다 강수량 시각(hh)14.0,1시간 최다 강수량 시각(hh)15.0,1시간 최다 강수량 시각(hh)16.0,1시간 최다 강수량 시각(hh)17.0,1시간 최다 강수량 시각(hh)18.0,1시간 최다 강수량 시각(hh)19.0,1시간 최다 강수량 시각(hh)20.0,1시간 최다 강수량 시각(hh)21.0,1시간 최다 강수량 시각(hh)22.0,1시간 최다 강수량 시각(hh)23.0,최소 상대습도 시각(hh)0,최소 상대습도 시각(hh)1,최소 상대습도 시각(hh)2,최소 상대습도 시각(hh)3,최소 상대습도 시각(hh)4,최소 상대습도 시각(hh)5,최소 상대습도 시각(hh)6,최소 상대습도 시각(hh)7,최소 상대습도 시각(hh)8,최소 상대습도 시각(hh)9,최소 상대습도 시각(hh)10,최소 상대습도 시각(hh)11,최소 상대습도 시각(hh)12,최소 상대습도 시각(hh)13,최소 상대습도 시각(hh)14,최소 상대습도 시각(hh)15,최소 상대습도 시각(hh)16,최소 상대습도 시각(hh)17,최소 상대습도 시각(hh)18,최소 상대습도 시각(hh)19,최소 상대습도 시각(hh)20,최소 상대습도 시각(hh)21,최소 상대습도 시각(hh)22,최소 상대습도 시각(hh)23,강수 계속시간(hr)
0,2018-01-01,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.766846,0.123827,0.048253,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-01-01,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.766846,0.123827,0.048253,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-01-01,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.766846,0.123827,0.048253,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-01-01,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.766846,0.123827,0.048253,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018-01-01,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.766846,0.123827,0.048253,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
sale_index = list(sale_df.columns[1:])
only_sale_df = sale_and_weather_df[sale_index]
sale_and_weather_df.drop(sale_index, axis = 1, inplace = True)
sale_and_weather_df = pd.concat([sale_and_weather_df, only_sale_df], axis = 1)

In [10]:
sale_and_weather_df.head()

,날짜,평균기온(°C),최저기온(°C),최고기온(°C),1시간 최다강수량(mm),일강수량(mm),평균 풍속(m/s),풍정합(100m),최소 상대습도(%),평균 상대습도(%),O3,PM10,PM25,최다풍향북북동,최다풍향북동,최다풍향동북동,최다풍향동,최다풍향동남동,최다풍향남동,최다풍향남남동,최다풍향남,최다풍향남남서,최다풍향남서,최다풍향서남서,최다풍향서,최다풍향서북서,최다풍향북서,최다풍향북북서,최다풍향북,최고기온 시각(hh)0,최고기온 시각(hh)1,최고기온 시각(hh)2,최고기온 시각(hh)3,최고기온 시각(hh)4,최고기온 시각(hh)5,최고기온 시각(hh)6,최고기온 시각(hh)7,최고기온 시각(hh)8,최고기온 시각(hh)9,최고기온 시각(hh)10,...,소분류_혼합견과,소분류_혼합곡,소분류_홍삼 간식,소분류_홍삼 분말/환,소분류_홍삼 음료,소분류_홍삼/인삼 제품,소분류_홍삼액/홍삼정,소분류_홍삼절편/홍삼정과,소분류_홍어,소분류_홍차,소분류_화장 비누,소분류_화장 퍼프,소분류_환자식,소분류_황토매트,소분류_회,소분류_휴대용 선풍기,소분류_흰우유,소분류_히터,20대 남성 판매량(개),20대 여성 판매량(개),30대 남성 판매량(개),30대 여성 판매량(개),40대 남성 판매량(개),40대 여성 판매량(개),50대 남성 판매량(개),50대 여성 판매량(개),60대 남성 판매량(개),60대 여성 판매량(개),일별 판매 합계량(개),일별 물품 선호도 점수,20대 남성 선호도 점수,20대 여성 선호도 점수,30대 남성 선호도 점수,30대 여성 선호도 점수,40대 남성 선호도 점수,40대 여성 선호도 점수,50대 남성 선호도 점수,50대 여성 선호도 점수,60대 남성 선호도 점수,60대 여성 선호도 점수
0,2018-01-01,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13.0,37.0,6.0,16.0,6.0,9.0,0.0,3.0,0.0,0.0,92.0,2.145570e-05,0.342493,0.329235,0.073506,0.079250,0.078651,0.054480,0.000000,0.042386,0.0,0.0
1,2018-01-01,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,4.0,3.0,6.0,1.0,7.0,1.0,1.0,0.0,0.0,25.0,4.259301e-06,0.000000,0.179295,0.185138,0.149704,0.066032,0.213452,0.135208,0.071171,0.0,0.0
2,2018-01-01,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,4.0,7.057713e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.184025,0.815975,0.000000,0.0,0.0
3,2018-01-01,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,2.0,13.0,16.0,7.0,15.0,1.0,3.0,0.0,0.0,60.0,1.146679e-05,0.049296,0.033299,0.297998,0.148285,0.171692,0.169899,0.050223,0.079309,0.0,0.0
4,2018-01-01,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,5.0,8.0,5.0,6.0,2.0,1.0,0.0,0.0,31.0,6.561077e-06,0.086154,0.029099,0.200312,0.129579,0.214333,0.118773,0.175548,0.046203,0.0,0.0


In [11]:
month_df = pd.DataFrame(pd.get_dummies(sale_and_weather_df['날짜'].dt.month))
month_df.columns = [str(i) + '월' for i in range(1, 13)]

In [12]:
sale_and_weather_df.drop(['날짜', '대분류_냉난방가전', '대분류_뷰티', '대분류_식품'], axis = 1, inplace = True)
sale_and_weather_df = pd.concat([month_df, sale_and_weather_df], axis = 1)

In [13]:
sale_and_weather_df.head()

,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월,평균기온(°C),최저기온(°C),최고기온(°C),1시간 최다강수량(mm),일강수량(mm),평균 풍속(m/s),풍정합(100m),최소 상대습도(%),평균 상대습도(%),O3,PM10,PM25,최다풍향북북동,최다풍향북동,최다풍향동북동,최다풍향동,최다풍향동남동,최다풍향남동,최다풍향남남동,최다풍향남,최다풍향남남서,최다풍향남서,최다풍향서남서,최다풍향서,최다풍향서북서,최다풍향북서,최다풍향북북서,최다풍향북,...,소분류_혼합견과,소분류_혼합곡,소분류_홍삼 간식,소분류_홍삼 분말/환,소분류_홍삼 음료,소분류_홍삼/인삼 제품,소분류_홍삼액/홍삼정,소분류_홍삼절편/홍삼정과,소분류_홍어,소분류_홍차,소분류_화장 비누,소분류_화장 퍼프,소분류_환자식,소분류_황토매트,소분류_회,소분류_휴대용 선풍기,소분류_흰우유,소분류_히터,20대 남성 판매량(개),20대 여성 판매량(개),30대 남성 판매량(개),30대 여성 판매량(개),40대 남성 판매량(개),40대 여성 판매량(개),50대 남성 판매량(개),50대 여성 판매량(개),60대 남성 판매량(개),60대 여성 판매량(개),일별 판매 합계량(개),일별 물품 선호도 점수,20대 남성 선호도 점수,20대 여성 선호도 점수,30대 남성 선호도 점수,30대 여성 선호도 점수,40대 남성 선호도 점수,40대 여성 선호도 점수,50대 남성 선호도 점수,50대 여성 선호도 점수,60대 남성 선호도 점수,60대 여성 선호도 점수
0,1,0,0,0,0,0,0,0,0,0,0,0,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13.0,37.0,6.0,16.0,6.0,9.0,0.0,3.0,0.0,0.0,92.0,2.145570e-05,0.342493,0.329235,0.073506,0.079250,0.078651,0.054480,0.000000,0.042386,0.0,0.0
1,1,0,0,0,0,0,0,0,0,0,0,0,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,4.0,3.0,6.0,1.0,7.0,1.0,1.0,0.0,0.0,25.0,4.259301e-06,0.000000,0.179295,0.185138,0.149704,0.066032,0.213452,0.135208,0.071171,0.0,0.0
2,1,0,0,0,0,0,0,0,0,0,0,0,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,4.0,7.057713e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.184025,0.815975,0.000000,0.0,0.0
3,1,0,0,0,0,0,0,0,0,0,0,0,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,2.0,13.0,16.0,7.0,15.0,1.0,3.0,0.0,0.0,60.0,1.146679e-05,0.049296,0.033299,0.297998,0.148285,0.171692,0.169899,0.050223,0.079309,0.0,0.0
4,1,0,0,0,0,0,0,0,0,0,0,0,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,5.0,8.0,5.0,6.0,2.0,1.0,0.0,0.0,31.0,6.561077e-06,0.086154,0.029099,0.200312,0.129579,0.214333,0.118773,0.175548,0.046203,0.0,0.0


In [15]:
# csv 파일로 저장
sale_and_weather_df.to_csv('sale_weather_data.csv', index = False)

In [16]:
X_df = sale_and_weather_df.iloc[:, :-22]

In [17]:
X_df.head()

,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월,평균기온(°C),최저기온(°C),최고기온(°C),1시간 최다강수량(mm),일강수량(mm),평균 풍속(m/s),풍정합(100m),최소 상대습도(%),평균 상대습도(%),O3,PM10,PM25,최다풍향북북동,최다풍향북동,최다풍향동북동,최다풍향동,최다풍향동남동,최다풍향남동,최다풍향남남동,최다풍향남,최다풍향남남서,최다풍향남서,최다풍향서남서,최다풍향서,최다풍향서북서,최다풍향북서,최다풍향북북서,최다풍향북,...,소분류_한방재료,소분류_한우육,소분류_해조류 다시마,소분류_해조류 미역,소분류_해초류,소분류_핸드워시/스크럽,소분류_핸드크림,소분류_향수세트,소분류_허브차,소분류_헛개/가시오가피,소분류_헤어 브러쉬,소분류_헤어매니큐어,소분류_헤어무스,소분류_헤어스타일링용 염색약,소분류_헤어스타일링용 펌제,소분류_헤어스타일링용 흑채,소분류_헤어스프레이,소분류_헤어에센스,소분류_헤어왁스,소분류_헤어젤,소분류_헤어케어세트,소분류_호박,소분류_혼합견과,소분류_혼합곡,소분류_홍삼 간식,소분류_홍삼 분말/환,소분류_홍삼 음료,소분류_홍삼/인삼 제품,소분류_홍삼액/홍삼정,소분류_홍삼절편/홍삼정과,소분류_홍어,소분류_홍차,소분류_화장 비누,소분류_화장 퍼프,소분류_환자식,소분류_황토매트,소분류_회,소분류_휴대용 선풍기,소분류_흰우유,소분류_히터
0,1,0,0,0,0,0,0,0,0,0,0,0,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
y_df = sale_and_weather_df.iloc[:, -22:]
y_df.head()

,20대 남성 판매량(개),20대 여성 판매량(개),30대 남성 판매량(개),30대 여성 판매량(개),40대 남성 판매량(개),40대 여성 판매량(개),50대 남성 판매량(개),50대 여성 판매량(개),60대 남성 판매량(개),60대 여성 판매량(개),일별 판매 합계량(개),일별 물품 선호도 점수,20대 남성 선호도 점수,20대 여성 선호도 점수,30대 남성 선호도 점수,30대 여성 선호도 점수,40대 남성 선호도 점수,40대 여성 선호도 점수,50대 남성 선호도 점수,50대 여성 선호도 점수,60대 남성 선호도 점수,60대 여성 선호도 점수
0,13.0,37.0,6.0,16.0,6.0,9.0,0.0,3.0,0.0,0.0,92.0,2.145570e-05,0.342493,0.329235,0.073506,0.079250,0.078651,0.054480,0.000000,0.042386,0.0,0.0
1,0.0,4.0,3.0,6.0,1.0,7.0,1.0,1.0,0.0,0.0,25.0,4.259301e-06,0.000000,0.179295,0.185138,0.149704,0.066032,0.213452,0.135208,0.071171,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,4.0,7.057713e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.184025,0.815975,0.000000,0.0,0.0
3,1.0,2.0,13.0,16.0,7.0,15.0,1.0,3.0,0.0,0.0,60.0,1.146679e-05,0.049296,0.033299,0.297998,0.148285,0.171692,0.169899,0.050223,0.079309,0.0,0.0
4,1.0,1.0,5.0,8.0,5.0,6.0,2.0,1.0,0.0,0.0,31.0,6.561077e-06,0.086154,0.029099,0.200312,0.129579,0.214333,0.118773,0.175548,0.046203,0.0,0.0


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size = 0.3, shuffle = True, random_state = 1004)

In [20]:
X_train.to_csv('X_train.csv', index = False)
X_test.to_csv('X_test.csv', index = False)
y_train.to_csv('y_train.csv', index = False)
y_test.to_csv('y_test.csv', index = False)